In [48]:
from pathlib import Path
import pandas as pd

path = Path('_data/wrds_bondret_filtered.parquet')
df = pd.read_parquet(path)


df['date'] = pd.to_datetime(df['date']).dt.normalize()
available_dates = df['date'].dropna().drop_duplicates().sort_values()
print('Available dates:')
print(available_dates.dt.strftime('%Y-%m-%d').to_list())

target_date = pd.Timestamp('2020-09-30')  # set to one value from Available dates
df = df[df['date'] == target_date].copy()
df.info()

Available dates:
['2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-05-31', '2005-06-30', '2005-07-31', '2005-08-31', '2005-09-30', '2005-10-31', '2005-11-30', '2005-12-31', '2006-01-31', '2006-02-28', '2006-03-31', '2006-04-30', '2006-05-31', '2006-06-30', '2006-07-31', '2006-08-31', '2006-09-30', '2006-10-31', '2006-11-30', '2006-12-31', '2007-01-31', '2007-02-28', '2007-03-31', '2007-04-30', '2007-05-31', '2007-06-30', '2007-07-31', '2007-08-31', '2007-09-30', '2007-10-31', '2007-11-30', '2007-12-31', '2008-01-31', '2008-02-29', '2008-03-31', '2008-04-30', '2

In [38]:
path2 = Path('_data/markit_cds.parquet')
df2 = pd.read_parquet(path2)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46851870 entries, 0 to 46851869
Data columns (total 7 columns):
 #   Column     Dtype         
---  ------     -----         
 0   date       datetime64[ns]
 1   ticker     string        
 2   redcode    string        
 3   parspread  Float64       
 4   tenor      string        
 5   country    string        
 6   year       int64         
dtypes: Float64(1), datetime64[ns](1), int64(1), string(4)
memory usage: 2.5 GB


In [39]:
path3 = Path('_data/RED_and_ISIN_mapping.parquet')
df3 = pd.read_parquet(path3)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11037 entries, 0 to 11036
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   redcode    11037 non-null  string
 1   ticker     11037 non-null  string
 2   obl_cusip  8000 non-null   string
 3   isin       11037 non-null  string
 4   tier       11037 non-null  string
dtypes: string(5)
memory usage: 431.3 KB


In [40]:
target_date = pd.Timestamp('2020-09-30')
df2_20200930 = df2[pd.to_datetime(df2['date']).dt.normalize() == target_date].copy()
df2_20200930.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7073 entries, 40152291 to 41945932
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       7073 non-null   datetime64[ns]
 1   ticker     7073 non-null   string        
 2   redcode    7073 non-null   string        
 3   parspread  7063 non-null   Float64       
 4   tenor      7073 non-null   string        
 5   country    7073 non-null   string        
 6   year       7073 non-null   int64         
dtypes: Float64(1), datetime64[ns](1), int64(1), string(4)
memory usage: 449.0 KB


In [47]:
# 1) Unique RED codes from df2_20200930
redcodes_20200930 = (
    df2_20200930[['redcode']]
    .dropna()
    .assign(redcode=lambda x: x['redcode'].astype(str).str.strip().str.upper())
    .query("redcode != '' and redcode != 'NAN'")
    .drop_duplicates()
    .reset_index(drop=True)
)

# 2) Clean RED->ISIN map from df3
redcode_isin_map = (
    df3[['redcode', 'isin']]
    .dropna(subset=['redcode', 'isin'])
    .assign(
        redcode=lambda x: x['redcode'].astype(str).str.strip().str.upper(),
        isin=lambda x: x['isin'].astype(str).str.strip().str.upper(),
    )
    .query("redcode != '' and redcode != 'NAN' and isin != '' and isin != 'NAN'")
    .drop_duplicates(subset=['redcode', 'isin'])
)

# 3) Keep only ISINs linked to RED codes present in df2_20200930
isins_from_redcodes_20200930 = (
    redcodes_20200930
    .merge(redcode_isin_map, on='redcode', how='left')
    .dropna(subset=['isin'])
    .drop_duplicates(subset=['redcode', 'isin'])
    .reset_index(drop=True)
)

print("unique redcodes:", redcodes_20200930['redcode'].nunique())
print("rows in redcode-isin dataframe:", len(isins_from_redcodes_20200930))
print("unique isins:", isins_from_redcodes_20200930['isin'].nunique())

isins_from_redcodes_20200930.head(30)


unique redcodes: 663
rows in redcode-isin dataframe: 2875
unique isins: 2817


,redcode,isin
0,1I96BB,US172474AC26
1,4CFFB9,US487836AS72
2,4CFFB9,XS0072952434
3,4CFFB9,US487836BD94
4,4CFFB9,US487836BU10
5,4CFFB9,US487836AT55
6,36AGC5,US337932AB30
7,36AGC5,US337932AC13
8,36AGC5,US337932AQ09
9,36AGC5,US337932AL12


In [49]:
# Normalize ISINs before comparison
df_isins = (
    df['isin']
    .dropna()
    .astype(str)
    .str.strip()
    .str.upper()
)
df_isins = set(df_isins[(df_isins != '') & (df_isins != 'NAN')])

mapped_isins = (
    isins_from_redcodes_20200930['isin']
    .dropna()
    .astype(str)
    .str.strip()
    .str.upper()
)
mapped_isins = set(mapped_isins[(mapped_isins != '') & (mapped_isins != 'NAN')])

overlap = mapped_isins.intersection(df_isins)

print("unique isins in isins_from_redcodes_20200930:", len(mapped_isins))
print("unique isins in df:", len(df_isins))
print("overlap unique isins:", len(overlap))
print("share of mapped isins found in df:", round(len(overlap) / len(mapped_isins), 4) if mapped_isins else 0)


unique isins in isins_from_redcodes_20200930: 2817
unique isins in df: 24681
overlap unique isins: 856
share of mapped isins found in df: 0.3039
